In [3]:
from dotenv import load_dotenv
import os
import pandas as pd
import re
import sqlalchemy
from sqlalchemy import create_engine

In [4]:
data_path = r'C:\Users\johnm\Code\RRC_Pipeline\assets\documents_20241122\PDQ_DSV\CSV\OG_COUNTY_CYCLE_DATA_TABLE.csv'
df = pd.read_csv(data_path, dtype={'COUNTY_NO': str})
print(df.head)

<bound method NDFrame.head of        COUNTY_NO  DISTRICT_NO  CYCLE_YEAR  CYCLE_MONTH  CYCLE_YEAR_MONTH  \
0            001            5        1993            1            199301   
1            001            5        1993            2            199302   
2            001            5        1993            3            199303   
3            001            5        1993            4            199304   
4            001            5        1993            5            199305   
...          ...          ...         ...          ...               ...   
231082       507            1        2024            9            202409   
231083       507            1        2024           10            202410   
231084       507            1        2024           10            202410   
231085       507            1        2024           11            202411   
231086       507            1        2024           11            202411   

        CNTY_OIL_PROD_VOL  CNTY_OIL_ALLOW  CNTY_OIL_ENDIN

C:\Users\johnm\AppData\Local\Temp\ipykernel_21608\1161791294.py:2: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path, dtype={'COUNTY_NO': str})


In [ ]:
# Data integrity checks
# print(df.columns)S
# print(len(df['DISCTRICT_NAME'].isna()))

Index(['COUNTY_NO', 'DISTRICT_NO', 'CYCLE_YEAR', 'CYCLE_MONTH',
       'CYCLE_YEAR_MONTH', 'CNTY_OIL_PROD_VOL', 'CNTY_OIL_ALLOW',
       'CNTY_OIL_ENDING_BAL', 'CNTY_GAS_PROD_VOL', 'CNTY_GAS_ALLOW',
       'CNTY_GAS_LIFT_INJ_VOL', 'CNTY_COND_PROD_VOL', 'CNTY_COND_LIMIT',
       'CNTY_COND_ENDING_BAL', 'CNTY_CSGD_PROD_VOL', 'CNTY_CSGD_LIMIT',
       'CNTY_CSGD_GAS_LIFT', 'CNTY_OIL_TOT_DISP', 'CNTY_GAS_TOT_DISP',
       'CNTY_COND_TOT_DISP', 'CNTY_CSGD_TOT_DISP', 'COUNTY_NAME',
       'DISTRICT_NAME', 'OIL_GAS_CODE', 'name', 'lng', 'lat'],
      dtype='object')


KeyError: 'DISCTRICT_NAME'

In [ ]:
# Columns of original data series
# print(df.columns)

# Counties that have oil production - duplicates exist
counties = df.loc[df['CNTY_COND_PROD_VOL'] > 0, 'COUNTY_NAME']

# Number of unique counties that have oil production
unique_counties = pd.unique(counties)
print("Number of unique counties that have oil production: ", len(unique_counties))

# How many counties in Texas are there?
print("How many counties in Texas are there? ", len(pd.unique(df['COUNTY_NAME'])))


In [ ]:
# Slider on year that you can see a heatmap of production over every year
# Create a map that has counties displayed in the map:

map_path = r'C:\Users\johnm\Code\RRC_Pipeline\assets\lat-long-us.csv'
new_csv = r'C:\Users\johnm\Code\RRC_Pipeline\assets\combined.csv'
df_map = pd.read_csv(map_path)

df_map = pd.read_csv(map_path, dtype={'fips_code': str}) # Read csv
df_map['is_fips_code'] = df_map['fips_code'].str[:2] == '48' # Captures the shortened fips code
df_map = df_map[df_map['is_fips_code']].reset_index(drop=True) # Creates a new dataframe without the entries that are not '48'
df_map['fips_code'] = df_map['fips_code'].str[2:] # Removing '48' from each entry for future comparisons
df_map = df_map[df_map['is_fips_code']].drop(columns=['is_fips_code']) # Dropping is_fips_code
df_map = df_map.set_index('fips_code') # Setting index to fips_code so merge will work on index
df['COUNTY_NO'] = df['COUNTY_NO'].astype(str) #Ensures the dataframe, df, has county_no's (fips codes) that are a string type.

# Left join - type of merge : how= left
# The key to join on is the 'COUNTY_NO' - lef_on
# The index from right dataframe is the join key - right_index
df = df.merge(df_map, left_on='COUNTY_NO', right_index=True, how = 'left') #Merges lookup_df with df

unmatched_rows = df[df['lat'].isnull()]['COUNTY_NO'] #Creates a dataframe that tracks null values of dataframe
unmatched_rows.to_csv("errors.txt", index=False) #Writes these null values to a csv

df.to_csv(new_csv, index = False) # Writes to the final csv

<bound method NDFrame.head of     fips_code       name         lng        lat
0         017     Bailey -102.829827  34.068603
1         329    Midland -102.031568  31.869041
2         251    Johnson  -97.366188  32.378814
3         171  Gillespie  -98.946769  30.318241
4         085     Collin  -96.572668  33.188227
..        ...        ...         ...        ...
249       197   Hardeman  -99.745730  34.290290
250       187  Guadalupe  -97.948465  29.583214
251       389     Reeves -103.693079  31.322833
252       489    Willacy  -97.657245  26.470406
253       225    Houston  -95.422003  31.317642

[254 rows x 4 columns]>


In [ ]:
# Get the average oil production for each county over 1993-2023 - change PBI to display average across all years

# Expenses or costs of production

# Does this data give anything on byproducts of wells or byproducts that can be used for other purposes?

# Find top 10 conties for 2023 production

# Find top 10 counties for 1993 production

# Number of wells in the permian over time

# Can I sample data using a model anadd

In [ ]:
print(df.head)
df.to_csv(new_csv, index=False)

In [ ]:
load_dotenv()
DATABASE_URL = os.getenv('DATABASE_URL')

try:
    df.to_sql(name="OG_COUNTY_CYCLE_DATA_TABLE", con=engine)
except Exception as e:
    print(e)

from sqlalchemy import text
with engine.connect() as conn:
    result = conn.execute(text('SELECT * FROM "OG_COUNTY_CYCLE_DATA_TABLE" LIMIT 10')).fetchall()

for index,row in enumerate(result):
    print(result[index])
    # print(len(result))


Table 'OG_COUNTY_CYCLE_DATA_TABLE' already exists.
(0, 1, 5, 1993, 1, 199301, 7355, None, None, 0, None, None, 0, None, None, 6347, None, None, None, None, None, None, 'ANDERSON', '05', 'O')
(1, 1, 5, 1993, 2, 199302, 6312, None, None, 0, None, None, 0, None, None, 4919, None, None, None, None, None, None, 'ANDERSON', '05', 'O')
(2, 1, 5, 1993, 3, 199303, 6222, None, None, 0, None, None, 0, None, None, 4973, None, None, None, None, None, None, 'ANDERSON', '05', 'O')
(3, 1, 5, 1993, 4, 199304, 6139, None, None, 0, None, None, 0, None, None, 4410, None, None, None, None, None, None, 'ANDERSON', '05', 'O')
(4, 1, 5, 1993, 5, 199305, 5785, None, None, 0, None, None, 0, None, None, 5961, None, None, None, None, None, None, 'ANDERSON', '05', 'O')
(5, 1, 5, 1993, 6, 199306, 4836, None, None, 0, None, None, 0, None, None, 5585, None, None, None, None, None, None, 'ANDERSON', '05', 'O')
(6, 1, 5, 1993, 7, 199307, 4779, None, None, 0, None, None, 0, None, None, 6179, None, None, None, None, None